In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import sys
import gc
from surprise import Reader, Dataset, SVD, KNNBasic
from surprise.model_selection import cross_validate
from ast import literal_eval
import pickle as pkl
from scipy import sparse
import gzip

In [272]:
metadata = pd.read_csv('archive/movies_metadata.csv', low_memory = False)

Remove empty titles

In [273]:
metadata = metadata[~metadata['title'].isnull()]

metadata = metadata.drop_duplicates('id')

metadata['id'] = metadata['id'].astype(int)

metadata.drop_duplicates('id', inplace = True)

metadata['popularity'] = metadata['popularity'].astype(float)

metadata['wr'] = (v / (v + m) * r) + (m / (v + m) * c)

Wighted Rating

In [274]:
v = metadata['vote_count']
r = metadata['vote_average']
c = metadata['vote_average'].mean()
m = metadata['vote_count'].quantile(.9)

Select only movies above 90% percentile on count of votes

In [275]:
metadata = metadata[
    metadata['vote_count'] > metadata['vote_count'].quantile(.9)
    ].sort_values('vote_count')

In [276]:
metadata.sort_values('wr', inplace = True, ascending = False)

In [277]:
print(metadata.shape)

metadata.head()

(4534, 25)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,wr
314,False,NaN,25000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",NaN,278,tt0111161,en,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,...,2.834147e+07,142.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fear can hold you prisoner. Hope can set you f...,The Shawshank Redemption,False,8.5,8358.0,8.079687
12481,False,"{'id': 263, 'name': 'The Dark Knight Collectio...",185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://thedarkknight.warnerbros.com/dvdsite/,155,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,...,1.004558e+09,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269.0,8.023661
2843,False,NaN,63000000,"[{'id': 18, 'name': 'Drama'}]",http://www.foxmovies.com/movies/fight-club,550,tt0137523,en,Fight Club,A ticking-time-bomb insomniac and a slippery s...,...,1.008538e+08,139.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Mischief. Mayhem. Soap.,Fight Club,False,8.3,9678.0,7.963320
834,False,"{'id': 230, 'name': 'The Godfather Collection'...",6000000,"[{'id': 18, 'name': 'Drama'}, {'id': 80, 'name...",http://www.thegodfather.com/,238,tt0068646,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",...,2.450664e+08,175.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,An offer you can't refuse.,The Godfather,False,8.5,6024.0,7.960223
292,False,NaN,8000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 80, 'n...",NaN,680,tt0110912,en,Pulp Fiction,"A burger-loving hit man, his philosophical par...",...,2.139288e+08,154.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Just because you are a character doesn't mean ...,Pulp Fiction,False,8.3,8670.0,7.932064


In [278]:
metadata['release_date'] = pd.to_datetime(metadata['release_date'])

metadata['year'] = metadata['release_date'].dt.year

rep = metadata.groupby('title')['title'].count()

count = [rep[i] for i in metadata['title']]

lamb = lambda a, n, y: a if n == 1 else f'{a}, {y:.0f}'

name = [lamb(a, n, y)  for a, n, y in zip(metadata['title'], count, metadata['release_date'].dt.year)]

metadata['title'] = name

In [279]:
credits = pd.read_csv('archive/credits.csv', index_col = 'id')

print(credits.shape)

credits.head(3)

(45476, 2)


,cast,crew
id,,
862,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
8844,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
15602,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."


In [280]:
credits.index = credits.index.astype(int)

credits = credits[credits.index.isin(metadata['id'].unique())]

credits = credits.reset_index().drop_duplicates(subset='id').set_index('id')

print(credits.shape)

(4534, 2)


In [281]:
keywords_df = pd.read_csv('archive/keywords.csv', index_col = 'id')

print(keywords_df.shape)

keywords_df.head(3)

(46419, 1)


,keywords
id,
862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."


In [282]:
keywords_df.index = keywords_df.index.astype(int)

keywords_df = keywords_df[keywords_df.index.isin(metadata['id'].unique())]

keywords_df = keywords_df.reset_index().drop_duplicates(subset='id').set_index('id')

print(keywords_df.shape)

(4534, 1)


### Add vote average

In [39]:
# rat_mod = metadata[['vote_average', 'popularity']].copy()

# rat_mod.index = metadata['id']

# rat_mod['vote_average'] = rat_mod['vote_average'] - rat_mod['vote_average'].mean()

# rat_mod['popularity'] = rat_mod['popularity'] - rat_mod['popularity'].mean()



In [297]:
rat_mod = pd.DataFrame(index = metadata['id'])

In [298]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        
        if len(names) > 5:
            names = names[:5]
            
        return names

    #Return empty list in case of missing/malformed data
    return []

### Get Genres

In [299]:
genres = metadata['genres'].apply(literal_eval).apply(get_list).apply(pd.Series).fillna('').stack().str.get_dummies().sum(level=0)

genres.index = metadata['id']

rat_mod = rat_mod.merge(genres, left_index = True, right_index = True)

print(rat_mod.shape)

(4534, 19)


### Get Collections

In [300]:
def get_collection(x):
    if isinstance(x, dict):
        
            
        return x.get('name')

    #Return empty list in case of missing/malformed data
    return []

In [301]:
collections = pd.get_dummies(metadata['belongs_to_collection'].fillna('{}').apply(literal_eval).apply(get_collection))

collections.index = metadata['id']

# Keep only with more then one value
collections = collections[collections.sum().sort_values().loc[lambda x : x > 1].index]

rat_mod = rat_mod.merge(collections, left_index = True, right_index = True, how = 'left').fillna(0)

print(rat_mod.shape)

(4534, 371)


### Get Production Companies

In [302]:
production_companies = metadata['production_companies'].apply(literal_eval).apply(get_list).apply(pd.Series)

production_companies = production_companies.fillna('').stack().to_frame()

production_companies = production_companies[
    production_companies[0].isin(production_companies.groupby(0).size().loc[lambda x: x > 10].index)
    ]

production_companies = production_companies[0].str.get_dummies().sum(level = 0)

rat_mod = rat_mod.merge(production_companies, left_index = True, right_index = True, how = 'left').fillna(0)

print(rat_mod.shape)

(4534, 556)


In [288]:
# production_companies = metadata['production_companies'].apply(literal_eval).apply(get_list)

# production_companies = production_companies.apply(pd.Series).fillna('').stack().str.get_dummies().sum(level=0)

# production_companies = production_companies[production_companies.sum().loc[lambda x: x > 10].index]

# production_companies.index = metadata['id']

# rat_mod = rat_mod.merge(production_companies, left_index = True, right_index = True, how = 'left').fillna(0)

# print(rat_mod.shape)

(4534, 556)


### Get Keywords

In [303]:
keywords = keywords_df['keywords'].apply(literal_eval).apply(get_list).apply(pd.Series)

keywords = keywords.fillna('').stack().to_frame()

keywords = keywords[keywords[0].isin(keywords.groupby(0).size().loc[lambda x: x > 5].index)]

keywords = keywords[0].str.get_dummies().sum(level = 0)

rat_mod = rat_mod.merge(keywords, left_index = True, right_index = True, how = 'left').fillna(0)

print(rat_mod.shape)

(4534, 1356)


In [48]:
# keywords_ = keywords['keywords'].apply(literal_eval).apply(get_list).apply(pd.Series)

# keywords_ = keywords_.apply(pd.Series).fillna('').stack().str.get_dummies().sum(level=0)

# keywords_ = keywords_[keywords_.sum().loc[lambda x: x > 3].index]

# rat_mod = rat_mod.merge(keywords_, left_index = True, right_index = True)

# print(rat_mod.shape)

(4534, 1789)


### Get Directors and Producers

In [304]:
def get_director(x):
    out = []
    director = False
    producer = False
    screen_play = False
    for i in x:
        if i['job'] == 'Director' and director == False: 
            if i['name'] != '':
                out.append(i['name']) 
                director = True
                
        elif i['job'] == 'Producer' and producer == False:
            out.append(i['name']) 
            producer = True
            
        elif i['job'] == 'Screenplay' and screen_play == False:
            out.append(i['name']) 
            screen_play = True

    return out

# or i['job'] == 'Producer' or i['job'] == 'Screenplay'

In [305]:
crew = credits['crew'].apply(literal_eval).apply(get_director).apply(pd.Series)

crew = crew.fillna('').stack().to_frame()

crew = crew[crew[0].isin(crew.groupby(0).size().loc[lambda x: x > 5].index)]

crew = crew[0].str.get_dummies().sum(level = 0)

crew = crew.astype(pd.SparseDtype()).info()

# rat_mod = rat_mod.merge(keywords, left_index = True, right_index = True, how = 'left').fillna(0)

print(rat_mod.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3577 entries, 862 to 248705
Columns: 437 entries, Adam McKay to Álvaro Augustín
dtypes: Sparse[float64, nan](437)
memory usage: 17.9 MB
(4534, 1356)


In [50]:
# crew = credits['crew'].apply(literal_eval)

# crew = crew.apply(get_director)

# crew = crew.apply(pd.Series).fillna('')

# crew = crew.stack().str.get_dummies().sum(level=0)

# crew = crew[crew.sum(level=0).sum().loc[lambda x: x > 3].index]

# rat_mod = rat_mod.merge(crew, left_index = True, right_index = True)

# print(rat_mod.shape)

(4534, 2612)


### Get Actors

In [306]:
actors = credits['cast'].apply(literal_eval).apply(get_list)

actors = actors.apply(pd.Series)

actors = actors.fillna('').stack().to_frame()

actors = actors[actors[0].isin(actors.groupby(0).size().loc[lambda x: x > 10].index)]

actors = actors[0].str.get_dummies().sum(level = 0)

In [307]:
rat_mod = rat_mod.merge(actors, left_index = True, right_index = True, how = 'left').fillna(0)

In [308]:
print(rat_mod.shape)

(4534, 1736)


In [51]:
# actors = credits['cast'].apply(literal_eval).apply(get_list)

# actors = actors.apply(pd.Series).fillna('').stack().str.get_dummies().sum(level=0)

# actors = actors[actors.sum().loc[lambda x: x > 4].index]

# rat_mod = rat_mod.merge(actors, left_index = True, right_index = True)

# print(rat_mod.shape)

KeyboardInterrupt: 

In [349]:
metadata_ratings = pd.merge(
    rat_mod.max(axis = 1 ).to_frame(),
    metadata,
    left_index = True,
    right_on = 'id',
    how = 'left'
    
    
)#.sort_values(0, ascending = False)

Calculate Similarity

In [53]:
# rat_mod.drop(['vote_average', 'popularity'], axis = 1, inplace = True)

In [416]:
similarity = cosine_similarity(rat_mod)

In [314]:
# similarity  = (similarity + 1.) / 2.

In [317]:
# similarity  = np.clip(similarity, 0, 1)

In [15]:
def get_recommendations(title, cosine_sim, indices, metadata, n = 10):
    if type(title) == list:
        ids = [indices[t] for t in title]
        
        sim_scores = list(enumerate(cosine_sim[ids].sum(axis = 0)))
        
    else:
        # Get the index of the movie that matches the title
        idx = indices[title]

        # Get the pairwsie similarity scores of all movies with that movie
        sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:n + 1]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata[['title', 0]].iloc[movie_indices]

In [16]:
indices = pd.Series(np.arange(metadata_ratings.shape[0]), index = metadata_ratings['title'])

In [204]:
indices['Interstellar']

4528

In [205]:
similarity[[2228, 50]].sum(axis = 0).shape

(3164,)

In [407]:
pd.DataFrame(similarity[0].todense()).values[0]

array([1.        , 0.18898224, 0.12598816, ..., 0.        , 0.10482848,
       0.        ])

In [17]:
get_recommendations(
    'Interstellar', 
    similarity, 
    indices, 
    metadata_ratings, n = 20
)

,title,0
30051,The Martian,1.0
17653,Another Earth,1.0
5276,Silent Running,1.0
4242,A.I. Artificial Intelligence,1.0
43645,Okja,1.0
8083,AVP: Alien vs. Predator,1.0
1512,Contact,1.0
1937,Flight of the Navigator,1.0
28811,Midnight Special,1.0
25865,Terminator Genisys,1.0


In [321]:
get_recommendations(
    'Star Wars', 
    similarity, 
    indices, 
    metadata_ratings, n = 20
)

,title,0
26555,Star Wars: The Force Awakens,1.0
1154,The Empire Strikes Back,1.0
7906,The Last Starfighter,1.0
1167,Return of the Jedi,1.0
12885,Star Wars: The Clone Wars,1.0
41489,Rogue One: A Star Wars Story,1.0
21922,Ender's Game,1.0
42079,Marvel One-Shot: Agent Carter,1.0
1785,Six Days Seven Nights,1.0
5040,"The Time Machine, 2002",1.0


In [322]:
get_recommendations(
    'The Empire Strikes Back', 
    similarity, 
    indices, 
    metadata_ratings
)

,title,0
1167,Return of the Jedi,1.0
256,Star Wars,1.0
26555,Star Wars: The Force Awakens,1.0
2514,Star Wars: Episode I - The Phantom Menace,1.0
7906,The Last Starfighter,1.0
41489,Rogue One: A Star Wars Story,1.0
10069,Star Wars: Episode III - Revenge of the Sith,1.0
12885,Star Wars: The Clone Wars,1.0
5244,Star Wars: Episode II - Attack of the Clones,1.0
21922,Ender's Game,1.0


In [323]:
get_recommendations(
    'The Dark Knight', 
    similarity, 
    indices, 
    metadata_ratings
)

,title,0
18252,The Dark Knight Rises,1.0
10122,Batman Begins,1.0
11415,Harsh Times,1.0
14911,Harry Brown,1.0
18109,The Devil's Double,1.0
14439,Cell 211,1.0
19395,Compliance,1.0
16813,Mesrine: Public Enemy #1,1.0
10810,Running Scared,1.0
10958,United 93,1.0


In [324]:
get_recommendations(
    'Avengers: Age of Ultron', 
    similarity, 
    indices, 
    metadata_ratings
)

,title,0
17818,"The Avengers, 2012",1.0
26567,Captain America: Civil War,1.0
15153,Iron Man 2,1.0
23053,Captain America: The Winter Soldier,1.0
26568,Doctor Strange,1.0
26562,Ant-Man,1.0
21941,Thor: The Dark World,1.0
12700,The Incredible Hulk,1.0
2527,Superman II,1.0
42079,Marvel One-Shot: Agent Carter,1.0


In [325]:
get_recommendations(
    "Harry Potter and the Philosopher's Stone", 
    similarity, 
    indices, 
    metadata_ratings, n = 20
)

,title,0
13893,Harry Potter and the Half-Blood Prince,1.0
5678,Harry Potter and the Chamber of Secrets,1.0
10554,Harry Potter and the Goblet of Fire,1.0
11927,Harry Potter and the Order of the Phoenix,1.0
17437,Harry Potter and the Deathly Hallows: Part 2,1.0
7725,Harry Potter and the Prisoner of Azkaban,1.0
1943,Hocus Pocus,1.0
5481,Spirited Away,1.0
16128,Harry Potter and the Deathly Hallows: Part 1,1.0
892,The Wizard of Oz,1.0


In [326]:
get_recommendations(
    'A View to a Kill', 
    similarity, 
    indices, 
    metadata_ratings, n = 20
)

,title,0
3511,On Her Majesty's Secret Service,1.0
3513,The Spy Who Loved Me,1.0
3517,The Man with the Golden Gun,1.0
2875,Live and Let Die,1.0
2873,For Your Eyes Only,1.0
3880,The Living Daylights,1.0
1640,Tomorrow Never Dies,1.0
2833,Dr. No,1.0
2832,From Russia with Love,1.0
7329,You Only Live Twice,1.0


In [327]:
get_recommendations(
    [
        "Harry Potter and the Philosopher's Stone", 
        'Harry Potter and the Chamber of Secrets', 
        'Harry Potter and the Prisoner of Azkaban',
        'Harry Potter and the Goblet of Fire'
    ], 
    similarity, 
    indices, 
    metadata_ratings
)

,title,0
5678,Harry Potter and the Chamber of Secrets,1.0
4766,Harry Potter and the Philosopher's Stone,1.0
10554,Harry Potter and the Goblet of Fire,1.0
7725,Harry Potter and the Prisoner of Azkaban,1.0
11927,Harry Potter and the Order of the Phoenix,1.0
17437,Harry Potter and the Deathly Hallows: Part 2,1.0
16128,Harry Potter and the Deathly Hallows: Part 1,1.0
5481,Spirited Away,1.0
16210,The Chronicles of Narnia: The Voyage of the Da...,1.0
9182,Kirikou and the Sorceress,1.0


In [328]:
get_recommendations(
    'Pulp Fiction', 
    similarity, 
    indices, 
    metadata_ratings
)

,title,0
22529,Reasonable Doubt,1.0
18034,Headhunters,1.0
6125,Basic,1.0
1844,The French Connection,1.0
3622,Shaft,1.0
23367,Blue Ruin,1.0
19395,Compliance,1.0
7271,Kill Bill: Vol. 2,1.0
1203,Touch of Evil,1.0
488,Menace II Society,1.0


In [333]:
get_recommendations(
    'The Shawshank Redemption', 
    similarity, 
    indices, 
    metadata_ratings, n = 20
)

,title,0
3080,Papillon,1.0
23057,Starred Up,1.0
14637,A Prophet,1.0
1596,Witness,1.0
10386,Green Street Hooligans,1.0
14052,Bronson,1.0
14424,The Bad Lieutenant: Port of Call - New Orleans,1.0
289,Leon: The Professional,1.0
12101,Gone Baby Gone,1.0
15186,Four Lions,1.0


In [736]:
rat_mod[rat_mod['Harry Potter Collection'] == 1]#['Harry Potter Collection']

,vote_average,popularity,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,...,Touchstone Pictures,TriStar Pictures,Twentieth Century Fox Film Corporation,United Artists,Universal Pictures,Village Roadshow Pictures,Walt Disney Pictures,Warner Bros.,Wild Bunch,Working Title Films
id,,,,,,,,,,,,,,,,,,,,,
767,0.753647,9.085895,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
675,0.753647,11.366472,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
674,0.853647,14.905230,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
672,0.753647,19.743624,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
673,1.053647,18.462448,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0
671,0.853647,28.189410,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


In [255]:
metadata_ratings[metadata['title'].str.contains('Gat')]

<ipython-input-255-af67fbc3e82a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metadata_ratings[metadata['title'].str.contains('Gat')]


,0,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
3237,1.0,False,NaN,38000000,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",NaN,622,tt0142688,en,The Ninth Gate,...,1999-08-24,58401898.0,133.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Every book has a life of its own.,The Ninth Gate,False,6.3,768.0
4096,1.0,False,NaN,68000000,"[{'id': 10752, 'name': 'War'}]",NaN,853,tt0215750,en,Enemy at the Gates,...,2001-03-13,96976270.0,131.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,Some Men Are Born To Be Heroes.,Enemy at the Gates,False,7.2,1023.0
1575,1.0,False,NaN,36000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 878, '...",NaN,782,tt0119177,en,Gattaca,...,1997-09-07,12532777.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,There is no gene for the human spirit.,Gattaca,False,7.5,1846.0
20910,1.0,False,NaN,105000000,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",NaN,64682,tt1343092,en,The Great Gatsby,...,2013-05-10,351040419.0,143.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Reserving judgments is a matter of infinite ho...,The Great Gatsby,False,7.3,3885.0


In [351]:
metadata_ratings.to_pickle('metadata.pkl')

In [6]:
metadata_ratings = pd.read_pickle('metadata.pkl')

In [331]:
with open('similarity.pkl', 'wb') as f:
    pkl.dump(similarity, f, )

In [4]:
with open('similarity.pkl', 'rb') as f:
        similarity =  pkl.load(f)

In [359]:
with open('similarity.npz', 'wb') as f:
    sparse.save_npz(f, similarity)

In [11]:
with gzip.open('similarity.pkl.gz', 'wb') as f:
    pkl.dump(similarity, f)

In [13]:
with gzip.open('similarity.pkl.gz', 'rb') as f:
    s = pkl.load(f)